<a href="https://colab.research.google.com/github/Cloblak/ad_photo_recommendation/blob/main/Model_Evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install cornac

In [ ]:
import os
import sys
import pickle
import numpy as np
import pandas as pd

import cornac
from cornac.utils import cache
from cornac.eval_methods import RatioSplit
from cornac.models import ItemKNN, AMR
from cornac.data import ImageModality

%tensorflow_version 1.x
import tensorflow as tf

print(f"System version: {sys.version}")
print(f"Cornac version: {cornac.__version__}")
print(f"Tensorflow version: {tf.__version__}")

SEED = 42
VERBOSE = True

In [ ]:
!git clone https://github.com/Cloblak/ad_photo_recommendation

In [4]:
%cd /content/ad_photo_recommendation

/content/ad_photo_recommendation


## Load Interaction Data

In [5]:
# Item-IDs
with open("data/interaction_data/img_ids", "rb") as fp:
  img_ids = pickle.load(fp)

# Image Features
with open("data/interaction_data/img_features", "rb") as fp:
  img_features = pickle.load(fp)

# Interaction/Feedback matrix
with open("data/interaction_data/interaction_data", "rb") as fp:
  interaction_data = pickle.load(fp)

## Model Evaluation

In [6]:
# Cosine similarity (Neighborhood Based)
cosine = ItemKNN(k=15, similarity="cosine", name="Cosine", verbose=VERBOSE, seed=SEED)

# Deep Learning model (Adversarial)
amr = AMR(k=15, k2=15, n_epochs=1, batch_size=10, learning_rate=0.1, lambda_w=0.1, lambda_b=0.1, lambda_e=0.0, seed=SEED)

item_image_modality = ImageModality(features=img_features, ids=img_ids, normalized=True)

ratio_split = RatioSplit(
    data=interaction_data,
    test_size=0.35,
    item_image=item_image_modality,
    verbose=VERBOSE,
    seed=SEED,
)

auc = cornac.metrics.AUC()

cornac.Experiment(eval_method=ratio_split, models=[cosine, amr], metrics=[auc]).run()

rating_threshold = 1.0
exclude_unknowns = True
---
Training data:
Number of users = 3
Number of items = 239
Number of ratings = 241
Max rating = 1.0
Min rating = 1.0
Global mean = 1.0
---
Test data:
Number of users = 3
Number of items = 18
Number of ratings = 18
Number of unknown users = 0
Number of unknown items = 0
---
Total users = 3
Total items = 239

[Cosine] Training started!


/usr/local/lib/python3.7/dist-packages/cornac/data/dataset.py:361: UserWarning: 16 duplicated observations are removed!
  warnings.warn("%d duplicated observations are removed!" % dup_count)


  0%|          | 0/239 [00:00<?, ?it/s]


[Cosine] Evaluation started!


Ranking:   0%|          | 0/3 [00:00<?, ?it/s]


[AMR] Training started!


Epoch 1/1:   0%|          | 0/25 [00:00<?, ?it/s]

Optimization finished!

[AMR] Evaluation started!


Ranking:   0%|          | 0/3 [00:00<?, ?it/s]


TEST:
...
       |    AUC | Train (s) | Test (s)
------ + ------ + --------- + --------
Cosine | 0.9397 |    0.0612 |   0.0501
AMR    | 0.8920 |    7.0172 |   0.0693

